In [3]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from threading import Thread
import time
import datetime


class MinimalIBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.reqId = 1
        self.contract = self.create_september_contract()  # September 2024 ES contract
        self.start_datetime = "20240620 00:00:00"  # Hardcoded start time when the June 2024 contract expires
        self.max_ticks = 1000
        self.last_tick = None  # To track the last tick received for overlap handling

    def create_september_contract(self):
        """Create the September 2024 ES futures contract."""
        contract = Contract()
        contract.symbol = "ES"
        contract.secType = "FUT"
        contract.exchange = "CME"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "202409"
        return contract

    def nextValidId(self, orderId):
        self.reqId = orderId
        self.request_historical_ticks()

    def request_historical_ticks(self):
        print(f"Requesting historical data starting from {self.start_datetime}")
        self.reqHistoricalTicks(self.reqId, self.contract, self.start_datetime, "", self.max_ticks, "TRADES", 1, True, [])

    def historicalTicks(self, reqId, ticks, done):
        print(f"Received {len(ticks)} ticks:")
        if ticks:
            if self.last_tick:
                # Check for overlap by comparing the first tick with the last tick from the previous batch
                assert ticks[0].time == self.last_tick.time, "Timestamps don't match!"
                print("First tick is the same as the last tick of the previous batch, discarding it.")
                ticks = ticks[1:]  # Discard the first tick if it's a duplicate

            for tick in ticks:
                timestamp = datetime.datetime.fromtimestamp(tick.time)
                print(f"Time: {timestamp}, Price: {tick.price}, Size: {tick.size}")

            # Update last tick received for overlap handling
            self.last_tick = ticks[-1]

            # Set the start time for the next request
            self.start_datetime = datetime.datetime.fromtimestamp(self.last_tick.time).strftime('%Y%m%d %H:%M:%S')

        if done:
            print("All historical tick data received.")
            self.disconnect()
        else:
            # Continue requesting more data
            self.reqId += 1
            self.request_historical_ticks()

    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson=None):
        print(f"Error: {errorCode} - {errorString}")


def main():
    app = MinimalIBApi()
    app.connect("127.0.0.1", 7497, clientId=1)

    # Start the socket in a thread to avoid blocking
    api_thread = Thread(target=app.run)
    api_thread.start()

    # Wait for data to be received
    time.sleep(10)

    # Disconnect the client
    app.disconnect()
    api_thread.join()


In [4]:
main()

Requesting historical data starting from 20240620 00:00:00
Error: 2104 - Market data farm connection is OK:usfarm.nj
Error: 2104 - Market data farm connection is OK:hfarm
Error: 2104 - Market data farm connection is OK:jfarm
Error: 2104 - Market data farm connection is OK:usfuture
Error: 2104 - Market data farm connection is OK:cashfarm
Error: 2104 - Market data farm connection is OK:usfarm
Error: 2104 - Market data farm connection is OK:eufarmnj
Error: 2106 - HMDS data farm connection is OK:euhmds
Error: 2106 - HMDS data farm connection is OK:fundfarm
Error: 2106 - HMDS data farm connection is OK:ushmds
Error: 2158 - Sec-def data farm connection is OK:secdefnj
Error: 2174 - Warning: You submitted request with date-time attributes without explicit time zone. Please switch to use yyyymmdd-hh:mm:ss in UTC or use instrument time zone, like US/Eastern. Implied time zone functionality will be removed in the next API release


In [5]:
!pip install databento

DEPRECATION: Loading egg at /opt/miniconda3/envs/ibkr/lib/python3.12/site-packages/ibapi-10.19.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.5/390.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 18.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 10.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 14.9 MB/s eta 0:00:00


In [29]:
import databento as db

client = db.Historical("db-dJcLuMNqLDSHyUc4xWFB5y8Prvub3")

params = {
    "dataset": "GLBX.MDP3",
    "symbols": "ESU4",
    "schema": "mbp-1",
    "start": "2024-01-01",
    "end": "2024-9-25"
}

cost = client.metadata.get_cost(**params)
record_count = client.metadata.get_record_count(**params)
billable_size = client.metadata.get_billable_size(**params)
size_gb = billable_size / 1024 / 1024 / 1024

print(f"Cost: {cost}")
print(f"Record count: {record_count:,}")
print(f"Billable size: {billable_size}")
print(f"Size in GB: {size_gb}")

Cost: 69.988660946488
Record count: 521,873,281
Billable size: 41749862480
Size in GB: 38.88258941471577


In [28]:
print("""
client.metadata.get_cost(**params)=62.694827377796
client.metadata.get_record_count(**params)=467486516
client.metadata.get_billable_size(**params)=37398921280
""")


client.metadata.get_cost(**params)=62.694827377796
client.metadata.get_record_count(**params)=467486516
client.metadata.get_billable_size(**params)=37398921280



In [ ]:
# order the data 
data = 